In [ ]:
#run this cell and then restart execution
!pip install accelerate
!pip install bitsandbytes
!pip install optimum
!pip install auto-gptq
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install -U gradio==3.32.0

In [2]:
import gradio as gr

In [ ]:
!pip install transformers
from transformers import AutoModelForCausalLM,AutoTokenizer
import torch
!pip install transformers huggingface_hub
#from huggingface_hub import notebook_login

#notebook_login()


In [4]:
mn = 'stabilityai/StableBeluga-7B'
#mn = "TheBloke/Llama-2-7b-Chat-GPTQ"

In [ ]:
model = AutoModelForCausalLM.from_pretrained(mn, device_map=0, load_in_8bit=True)

#model = AutoModelForCausalLM.from_pretrained(mn, device_map=0, torch_dtype=torch.float16)

In [52]:
sb_sys = "### System:\nYou are a AI assistant in my car, that follows instructions extremely well. Help as much as you can. Answer questions concisely and effectively with the requested information.\n\n"


In [42]:
def gen(p, maxlen=15, sample=True):
    toks = tokr(p, return_tensors="pt")
    res = model.generate(**toks.to("cuda"), max_new_tokens=maxlen, do_sample=sample).to('cpu')
    return tokr.batch_decode(res)

In [43]:
tokr = AutoTokenizer.from_pretrained(mn)

In [44]:
#to have a prompt corresponding to the specific format required by the fine-tuned model Stable Beluga
def mk_prompt(user, syst=sb_sys): return f"{syst}### User: {user}\n\n### Assistant:\n"

In [45]:
complete_answer= ''

In [46]:
#attempt to get user location

import requests

response = requests.get("http://ip-api.com/json/")
data = response.json()
print(data['city'], data['lat'], data['lon'])
city= data['city']
lat = data['lat']
lon = data['lon']


The Dalles 45.5945 -121.1786


In [51]:
import re
model_answer= ''
general_context= f'I am in my car in {city}. My exact position is latitude {lat}, longitude {lon}. I can move with my car to reach a destination'
# Define the initial state with some initial context.
initial_state = {'context': general_context}
pattern = r"Assistant:\\n(.*?)</s>"
#The api key for weather (and only for weather) is a08df3f6b6264a1facc135137230311

In [48]:
def car_answer_only(complete_answer, general_context):
    match = re.search(pattern, complete_answer, re.DOTALL)

    if match:
        # Extracting the text
        model_answer = match.group(1)
    else:
        model_answer = "There has been an error with the generated response."

    general_context +=  model_answer
    return (model_answer, general_context)
#print(model_answer)

In [49]:
def FnAnswer(general_context, ques, state):
    # Initialize state if it is None
    if state is None:
        state = {}
        state['context'] = general_context


    question=f"""Answer the question with the help of the provided context.

    ## Context

    {general_context} .

    ## Question

    {ques}"""

    complete_answer = str(gen(mk_prompt(question), 300))

    model_answer, general_context= car_answer_only(complete_answer, general_context)

    state['context'] = state['context'] + ". " + model_answer


    return model_answer, state['context'], state

In [50]:
initial_context= initial_state['context']
# Create the Gradio interface.
iface = gr.Interface(
    fn=FnAnswer,
    inputs=[
        gr.Textbox(value=initial_context, visible=False),
        gr.Textbox(lines=2, placeholder="Type your message here..."),
        gr.State()  # This will keep track of the context state across interactions.
    ],
    outputs=[
        gr.Textbox(),
        gr.Textbox(visible=False),
        gr.State()
    ]
)
# Launch the interface.
iface.launch(debug=True, share=True)
#contextual=gr.Textbox(value=general_context, visible=False)
#demo = gr.Interface(fn=FnAnswer, inputs=[contextual,"text"], outputs=["text", contextual])

#demo.launch()

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://35a817dd1b018474f4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://35a817dd1b018474f4.gradio.live


In [ ]:
#weather API

import requests

def get_weather_with_weatherapi(api_key:str, city_name:str):
    # The endpoint URL provided by WeatherAPI
    url = f"http://api.weatherapi.com/v1/current.json?key={api_key}&q={city_name}&aqi=no"

    # Make the API request
    response = requests.get(url)

    if response.status_code == 200:
        # Parse the JSON response
        weather_data = response.json()

        # Extracting the necessary pieces of data
        location = weather_data['location']['name']
        region = weather_data['location']['region']
        country = weather_data['location']['country']
        time = weather_data['location']['localtime']
        temperature_c = weather_data['current']['temp_c']
        condition_text = weather_data['current']['condition']['text']
        wind_mph = weather_data['current']['wind_mph']
        humidity = weather_data['current']['humidity']
        feelslike_c = weather_data['current']['feelslike_c']

        # Formulate the sentences
        weather_sentences = (
            f"The current weather in {location}, {region}, {country} is {condition_text} "
            f"with a temperature of {temperature_c}°C that feels like {feelslike_c}°C. "
            f"Humidity is at {humidity}%. "
            f"Wind speed is {wind_mph} mph."
            f"the local time is {time}"
        )
        return weather_sentences
    else:
        # Handle errors
        return f"Failed to get weather data: {response.status_code}, {response.text}"

# Replace 'YOUR_API_KEY' with your actual API key from WeatherAPI and 'CITY_NAME' with your city
api_key = 'a08df3f6b6264a1facc135137230311'
city_name = 'Las Vegas'
weather_info = get_weather_with_weatherapi(api_key, city_name)
print(weather_info)


The current weather in Las Vegas, Nevada, United States of America is Sunny with a temperature of 9.4°C that feels like 9.6°C. Humidity is at 44%. Wind speed is 3.8 mph.the local time is 2023-11-23 8:20


In [ ]:
#traffic API

import requests

api_key = 'YOUR_API_KEY'  # Replace with your actual TomTom API key
version_number = '1'  # Check the latest API version in the documentation

# TomTom Traffic API endpoint example for traffic incidents
traffic_incidents_endpoint = f"https://api.tomtom.com/traffic/services/5/incidentDetails?bbox=4.8854592519716675%2C52.36934334773164%2C4.897883244144765%2C52.37496348620152&fields=%7Bincidents%7Btype%2Cgeometry%7Btype%2Ccoordinates%7D%2Cproperties%7BiconCategory%7D%7D%7D&language=en-GB&categoryFilter=0%2C1%2C2%2C3%2C4%2C5%2C6%2C7%2C8%2C9%2C10%2C11%2C14&timeValidityFilter=present&key=FY0suJKC0xfktJmHjnNOGLXRmSyvS6zi"



response = requests.get(traffic_incidents_endpoint, params=params)

if response.status_code == 200:
    # Process the response if successful
    traffic_data = response.json()
    # Implement your logic to incorporate traffic_data into your language model
    print(traffic_data)
else:
    print(f"Error: {response.status_code}")

{'incidents': [{'type': 'Feature', 'properties': {'iconCategory': 8}, 'geometry': {'type': 'LineString', 'coordinates': [[4.8869043181, 52.373369829], [4.8868144641, 52.3722794802]]}}, {'type': 'Feature', 'properties': {'iconCategory': 8}, 'geometry': {'type': 'LineString', 'coordinates': [[4.8893679271, 52.3715365312], [4.8897166142, 52.3715780851]]}}, {'type': 'Feature', 'properties': {'iconCategory': 8}, 'geometry': {'type': 'LineString', 'coordinates': [[4.8905708978, 52.3716625745], [4.8902865837, 52.371635759], [4.8897166142, 52.3715780851], [4.8893679271, 52.3715365312]]}}, {'type': 'Feature', 'properties': {'iconCategory': 6}, 'geometry': {'type': 'LineString', 'coordinates': [[4.8924296687, 52.3698788984], [4.8924739251, 52.3699499828], [4.8925704846, 52.3701484964], [4.8926348577, 52.3703295068], [4.8927394638, 52.3707090788], [4.8927609215, 52.3707935698], [4.892793108, 52.3708954603], [4.8928293178, 52.3710402868], [4.8928346822, 52.3710644415], [4.8928601632, 52.3711851639

In [ ]:
#schema of function, may be useful to use python functions in the model (not sure)
from pydantic import create_model
import inspect, json
from inspect import Parameter



def schema(f):
    kw = {n:(o.annotation, ... if o.default==Parameter.empty else o.default)
          for n,o in inspect.signature(f).parameters.items()}
    s = create_model(f'Input for `{f.__name__}`', **kw).schema()
    return dict(name=f.__name__, description=f.__doc__, parameters=s)

In [ ]:
schema(get_weather_with_weatherapi)

{'name': 'get_weather_with_weatherapi',
 'description': None,
 'parameters': {'title': 'Input for `get_weather_with_weatherapi`',
  'type': 'object',
  'properties': {'api_key': {'title': 'Api Key', 'type': 'string'},
   'city_name': {'title': 'City Name', 'type': 'string'}},
  'required': ['api_key', 'city_name']}}